In [45]:
# !pip install typing_extensions==4.7.1
# import typing_extensions
# from importlib import reload
# reload(typing_extensions)

In [46]:
# hide
!pip install deepspeed
!pip install --upgrade wandb
import copy
from functools import partial
import multiprocessing as mp
from pathlib import Path
from typing import Any, Callable, List, Tuple
import os
from PIL import Image
import PIL.ImageOps
from PIL import ImageFile
import math
import json
import time
ImageFile.LOAD_TRUNCATED_IMAGES = True



from deepspeed.ops.adam import FusedAdam
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import train_test_split
import torch
from torch.distributions.bernoulli import Bernoulli
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import io, transforms
# from torchvision.utils import Image, ImageDraw
from torchvision.transforms.functional import to_pil_image
from tqdm.auto import tqdm


In [47]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive')

import utils1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
def is_image_file(filename):
#    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))
    return filename.lower().endswith(('.png', '.jpg', '.jpeg'))

In [49]:
# Image parameters
TRAIN_FILES = "/kaggle/input/spamv1/spamimg_train/Products"
IMAGE_SIZE = 256
PATCH_SIZE = 16
ZERO_PCT = 0.1
PATCHES_PER_ROW = (IMAGE_SIZE // PATCH_SIZE)
NUM_PATCHES = PATCHES_PER_ROW ** 2
RGB_CHANNELS = 3
NUM_PIXELS = PATCH_SIZE ** 2 * RGB_CHANNELS
VALID_IMAGES = 5
TOPK = 5

# Training parameters
BATCH_SIZE = 32
EPOCHS = 5
LR = 1e-4

# Transformer parameters
N_HEADS = 8
N_LAYERS = 6

# Update constants
TEMPERATURE_S = 0.1
TEMPERATURE_T = 0.05
CENTER_MOMENTUM = 0.9
TEACHER_MOMENTUM = 0.995

In [50]:
class CFG:
    BATCH_SIZE = 16
    num_workers = 4
    WeightSampler = True
    seed = 122

    model_name = "swin_v2_t"
    pretrained = True

    learning_rate = 5e-6
    weight_decay = 1e-6

    embedding_size = 512
    loss_function = "" # ContrastiveLoss else BCE loss
    replace_fc = False

    threshold = 0.5
    # margin for contrastive loss
    margin = 9.5

    EPOCHS = 10
    early_stopping = 25

    model_path = "/kaggle/input/bigdataset-models" # Dir to load model to test
    output_dir = "/kaggle/working/" # Dir to save model after train
    mode = "" # Training or Inference else All
                        #if mode is All set model_path and output_dir to ""
    # Dataset
    ratio = 0.80
    SKFold = True
    train_dir = "/kaggle/input/spamv2/SelectedP6"
    train_csv = "/kaggle/input/dataset-csvs/v5/paired_train_big_10000_1_1_.csv"

    test_dir = "/kaggle/input/spamv2/SelectedP6"
    test_csv = "/kaggle/input/csvfile/balanced_test_part6.csv"

    online_pairing = False
    neg_ratio = 1
    pos_ratio = 1

In [51]:
class ImageData(Dataset):
    def __init__(self, files: List[str]):
        self.files = files

        self.to_tensor = transforms.PILToTensor()
        self.randcrop_big = transforms.RandomResizedCrop((IMAGE_SIZE, IMAGE_SIZE), scale=(0.5, 1.0))
        self.randcrop_small = transforms.RandomResizedCrop((IMAGE_SIZE, IMAGE_SIZE))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        img = Image.open((self.files[i]))
        img = img.convert('RGB')
        #img = self.to_tensor(img)

        img1 = self.randcrop_big(img)
        img2 = self.randcrop_small(img)

        # img1 = self.to_tensor(img1)
        # img2 = self.to_tensor(img2)

        if  self.to_tensor(img).shape[0] == 1:
            img1 = torch.cat([img1]*3)
            img2 = torch.cat([img2]*3)

        return self.to_tensor(img1), self.to_tensor(img2)

class CollateFn:
    def reshape(self, batch):
        patches = torch.stack(batch)\
                    .unfold(2, PATCH_SIZE, PATCH_SIZE)\
                    .unfold(3, PATCH_SIZE, PATCH_SIZE)

        num_images = len(patches)
        patches = patches.reshape(
            num_images,
            RGB_CHANNELS,
            NUM_PATCHES,
            PATCH_SIZE,
            PATCH_SIZE
        )
        patches.transpose_(1, 2)

        return patches.reshape(num_images, NUM_PATCHES, -1) / 255.0 - 0.5

    def __call__(
        self, batch: List[Tuple[torch.Tensor, torch.Tensor]]
    ) -> Tuple[torch.FloatTensor, torch.FloatTensor]:
        x1, x2 = zip(*batch)

        return self.reshape(x1), self.reshape(x2)

In [56]:
# hide
class ImageOriginalData(Dataset):
    def __init__(self, files: List[str]):
        self.files = files

        self.to_tensor = transforms.PILToTensor()
        self.resize = transforms.Resize((IMAGE_SIZE, IMAGE_SIZE))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        img = Image.open((self.files[i]))
        img = img.convert('RGB')
        img = self.to_tensor(img)

        if  img.shape[0] == 1:
            img = torch.cat([img]*3)

        return self.resize(img)

class CollateSingleImage(CollateFn):
    def __call__(
        self, batch: List[torch.Tensor]
    ) -> torch.FloatTensor:
        return self.reshape(batch)



In [57]:
files = [str(file) for file in Path(TRAIN_FILES).rglob('*') if is_image_file(str(file))]

print(len(files))
train_files, valid_files = train_test_split(files,test_size=0.15, random_state=42)



0


ValueError: With n_samples=0, test_size=0.15 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [58]:
train_data = ImageData(train_files)
train_dl = DataLoader(
    train_data,
    BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=CFG.num_workers,
    pin_memory=True,
    collate_fn=CollateFn()
)

valid_data = ImageOriginalData(valid_files)
valid_dl = DataLoader(
    valid_data,
    BATCH_SIZE*2,
    shuffle=False,
    drop_last=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    collate_fn=CollateSingleImage()
)

NameError: name 'train_files' is not defined

In [59]:
# Function to save checkpoint
def save_checkpoint(model, optimizer, filename):
    print("==> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

# Function to load checkpoint
def load_checkpoint(checkpoint_file):
    print("==> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=CFG.device)
    model.load_state_dict(checkpoint["state_dict"], assign=True)
    optimizer.load_state_dict(checkpoint["optimizer"])
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(CFG.device)


def save_pkl(pkl_list,filename):
    with open(filename, 'wb') as f:       #this will save the list as "results.pkl" which you can load in later
                pkl.dump(pkl_list, f)          #as a list to python


def load_pkl(filename):
    with open(filename, 'rb') as f:       #this will save the list as "results.pkl" which you can load in later
            pkl_list =  pkl.load(f)
    return pkl_list


In [60]:
x, y = next(iter(train_dl))
x2 = next(iter(valid_dl))
(x.shape, y.shape), (x2.shape)

NameError: name 'train_dl' is not defined

In [ ]:
class Model(nn.Module):
    def __init__(self, d_model, n_head, n_layers):
        super().__init__()
        # transformer
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        # positional embedding
        w_pos = torch.randn(NUM_PATCHES, d_model) / d_model ** 0.5
        cls_token = torch.randn(1, d_model) / d_model ** 0.5
        self.register_parameter("pos_embed", nn.Parameter(w_pos))
        self.register_parameter("cls_token", nn.Parameter(cls_token))

        # pixel projection
        self.linear = nn.Linear(2 * d_model, d_model)
        self.norm1 = nn.LayerNorm(2 * d_model, elementwise_affine=False)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        batch_size = len(x)
        position = torch.stack([self.pos_embed] * batch_size)
        x = torch.cat([x, position], dim=-1)
        pixel_proj = self.norm2(F.relu(self.linear(self.norm1(x))))
        batched_cls_token = torch.stack([self.cls_token]*batch_size)
        cls_x = torch.cat([batched_cls_token, pixel_proj], dim=1)

        cls_x.transpose_(0, 1)
        return F.normalize(self.encoder(cls_x)[0, ...], dim=-1)

In [ ]:
class HLoss:
    def __init__(self, temperature_t: float, temperature_s: float):
        self.temperature_t = temperature_t
        self.temperature_s = temperature_s

    def __call__(
        self,
        t: torch.FloatTensor,
        s: torch.FloatTensor,
        center: torch.FloatTensor
    ) -> torch.FloatTensor:
        t = F.softmax((t.detach() - center.cuda()) / self.temperature_t, dim=1)

        log_s = F.log_softmax(s / self.temperature_s, dim=1)

        return -(t * log_s).sum(dim=1).mean()

In [ ]:
# hide
resize = transforms.Resize((IMAGE_SIZE, IMAGE_SIZE))

def get_closest(embedding: torch.FloatTensor, i: int):
    similarity = embedding @ embedding[i,:].T
    scores, idx = similarity.topk(TOPK)
    return scores.cpu().numpy(), idx.cpu().numpy()

def get_closest_wandb_images(embedding: torch.FloatTensor, i: int, files: List[str]):
    main_img = to_pil_image(resize(io.read_image(files[i])))
    closest_imgs = [wandb.Image(main_img)]

    scores, idx = get_closest(embedding, i)

    for i, score in zip(idx, scores):
        img = to_pil_image(resize(io.read_image(files[i])))
        closest_imgs.append(
            wandb.Image(img, caption=f"{score:.4f}")
        )

    return closest_imgs

In [ ]:
class DINOLossT(nn.Module):
    def __init__(self, out_dim, ncrops, warmup_teacher_temp, teacher_temp,
                 warmup_teacher_temp_epochs, nepochs, student_temp=0.1,
                 center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.center_momentum = center_momentum
        self.ncrops = ncrops
        self.register_buffer("center", torch.zeros(1, out_dim))
        # we apply a warm up for the teacher temperature because
        # a too high temperature makes the training instable at the beginning
        self.teacher_temp_schedule = np.concatenate((
            np.linspace(warmup_teacher_temp,
                        teacher_temp, warmup_teacher_temp_epochs),
            np.ones(nepochs - warmup_teacher_temp_epochs) * teacher_temp
        ))

    def forward(self, student_output, teacher_output, epoch):
        """
        Cross-entropy between softmax outputs of the teacher and student networks.
        """
        student_out = student_output / self.student_temp
        student_out = student_out.chunk(self.ncrops)

        # teacher centering and sharpening
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss = 0
        n_loss_terms = 0
        for iq, q in enumerate(teacher_out):
            for v in range(len(student_out)):
                if v == iq:
                    # we skip cases where student and teacher operate on the same view
                    continue
                loss = torch.sum(-q * F.log_softmax(student_out[v], dim=-1), dim=-1)
                total_loss += loss.mean()
                n_loss_terms += 1
        total_loss /= n_loss_terms
        self.update_center(teacher_output)
        return total_loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update center used for teacher output.
        """
        batch_center = torch.sum(teacher_output, dim=0, keepdim=True)
        dist.all_reduce(batch_center)
        batch_center = batch_center / (len(teacher_output) * dist.get_world_size())

        # ema update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)



In [ ]:
class DinoLoss(nn.Module):
    def __init__(
        self,
        teacher: nn.Module,
        student: nn.Module,
        loss_fn: Callable,
        dim: int,
        center_momentum: float,
        param_momentum: float,
    ):
        super().__init__()

        self.teacher = teacher
        self.student = student
        self.loss_fn = loss_fn

        self.c_mom = center_momentum
        self.p_mom = param_momentum
        self.register_buffer("center", torch.zeros((1, dim)).float())


    def forward(self,t1,t2,s1,s2):
        loss = self.loss_fn(t1, s2, self.center) + self.loss_fn(t2, s1, self.center)
        emperical_center = F.normalize(
            torch.cat([t1, t2]).mean(dim=0, keepdims=True),
            dim=-1,
        )
        self.update_center(emperical_center)
        return loss

    def update_center(self,emperical_center):
        self.center = F.normalize(
            self.c_mom * self.center.cuda() + (1 - self.c_mom) * emperical_center,
            dim=-1,
        )
        for s_p, t_p in zip(self.student.parameters(), self.teacher.parameters()):
            t_p.data = self.p_mom * t_p.data + (1 - self.p_mom) * s_p.data


In [ ]:
class LightningModel(pl.LightningModule):
    def __init__(
        self,
        teacher: nn.Module,
        lr: float,
        loss_fn: Callable,
        valid_files: List[str],
        dim: int,
        center_momentum: float,
        param_momentum: float,
    ):
        super().__init__()
        self.teacher = teacher
        self.student = copy.deepcopy(teacher)
        self.lr = lr
        self.loss_fn = loss_fn
        self.c_mom = center_momentum
        self.p_mom = param_momentum
        self.register_buffer("center", torch.zeros((1, dim)).float())
        self.valid_files = valid_files
        self.validation_step_outputs = []

        for p in self.teacher.parameters():
            p.requires_grad = False

    def loss_calculation(
        self,
        batch: Tuple[torch.FloatTensor, torch.FloatTensor],
    ) -> torch.FloatTensor:
        x1, x2 = batch

        s1, s2 = self.student(x1), self.student(x2)
        t1, t2 = self.teacher(x1), self.teacher(x2)

        loss = self.loss_fn(t1, s2, self.center) + self.loss_fn(t2, s1, self.center)

        emperical_center = F.normalize(
            torch.cat([t1, t2]).mean(dim=0, keepdims=True),
            dim=-1,
        )
        return loss, emperical_center

    def training_step(
        self, batch: Tuple[torch.FloatTensor, torch.FloatTensor], *args: List[Any]
    ) -> torch.Tensor:
        loss, emperical_center = self.loss_calculation(batch)
        self.log(name="Training loss", value=loss, on_step=True, on_epoch=True)

        self.center = F.normalize(
            self.c_mom * self.center + (1 - self.c_mom) * emperical_center,
            dim=-1,
        )
        for s_p, t_p in zip(self.student.parameters(), self.teacher.parameters()):
            t_p.data = self.p_mom * t_p.data + (1 - self.p_mom) * s_p.data
        return loss

    def validation_step(self, images: torch.FloatTensor, *args: List[Any]) -> None:
        return self.validation_step_outputs.append(self.teacher(images))

    def on_validation_epoch_end(self):
        valid_embeds = torch.cat([pred for pred in self.validation_step_outputs])
        columns = ["image"] + [f"closest_{i+1}" for i in range(TOPK)]
        indices = np.random.choice(len(self.valid_files), VALID_IMAGES, replace=False)
        rows = [get_closest_wandb_images(valid_embeds, i, self.valid_files) for i in indices]
        table = wandb.Table(data=rows, columns=columns)
        self.logger.experiment.log({f"epoch {self.current_epoch} results": table})
        self.validation_step_outputs.clear()

    def on_after_backward(self):
        if self.trainer.global_step % 50 == 0:  # don't make the tf file huge
            global_step = self.trainer.global_step
            for name, param in self.student.named_parameters():
                if "weight" in name and not "norm" in name and param.requires_grad:
                    self.logger.experiment.log(
                        {f"{name}_grad": wandb.Histogram(param.grad.cpu())}
                    )

    def configure_optimizers(self) -> torch.optim.Optimizer:
        return FusedAdam(self.student.parameters(), lr=self.lr)

In [ ]:

def training_one_epoch(train_dataloader, epoch, loss_fn, student, teacher, optimizer):
#                       teacher_without_ddp, lr_schedule, momentum_schedule, fp16_scaler, args):

        student.train()
        teacher.train()
        losses=[]
        metric_logger = utils.MetricLogger(delimiter="  ")
        header = 'Epoch: [{}/{}]'.format(epoch, CFG.EPOCHS)

        for p in teacher.parameters():
            p.requires_grad = False
        for i, data in enumerate(metric_logger.log_every(train_dataloader, 10, header)):
        #for i, data in enumerate(loop,0):
            img1, img2 = data
            img1, img2 = img1.to(device), img2.to(device)
            for param in teacher.parameters():
              param.grad = None
            for param in student.parameters():
              param.grad = None
            s1, s2 = student(img1), student(img2)
            t1, t2 = teacher(img1), teacher(img2)
            loss = loss_fn(t1,t2,s1,s2)
            if not math.isfinite(loss.item()):
                print("Loss is {}, stopping training".format(loss.item()), force=True)
                sys.exit(1)

            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            mean_loss =  np.mean(losses)
            # loop.set_description(f"Epoch [{epoch}/{CFG.EPOCHS}]")
            # loop.set_postfix(Loss=mean_loss)

            metric_logger.update(loss=loss.item())
            metric_logger.update(lr=optimizer.param_groups[0]["lr"])
            metric_logger.update(wd=optimizer.param_groups[0]["weight_decay"])

        print("Averaged stats:", metric_logger)
        return {k: meter.global_avg for k, meter in metric_logger.meters.items()}



Thanks to PyTorch Lightning and WandB I can easily do half precision training and log the results to a beautiful dashboard, with results in the link below.

In [ ]:
!mkdir /kaggle/working/logs
teacher = Model(NUM_PIXELS, N_HEADS, N_LAYERS).to(device)
student = copy.deepcopy(teacher)
optimizer = torch.optim.AdamW(student.parameters(), lr=LR)
h_loss = HLoss(TEMPERATURE_T, TEMPERATURE_S)
dino_loss = DinoLoss(
    teacher,
    student,
    h_loss,
    NUM_PIXELS,
    CENTER_MOMENTUM,
    TEACHER_MOMENTUM,
)

#logger = WandbLogger("DINO", "/kaggle/working/logs/", project="DINO")


In [ ]:
def training_model(epochs= CFG.EPOCHS):
  train_loss=[]
  eval_loss=[]
  for epoch in range(0,epochs):
        train_stats = training_one_epoch(train_dataloader=train_dl, epoch=epoch, loss_fn=dino_loss, student=student, teacher=teacher, optimizer=optimizer)
        # ============ writing logs ... ============
        save_dict = {
            'student': student.state_dict(),
            'teacher': teacher.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            #'args': args,
            'dino_loss': dino_loss.state_dict(),
        }
        utils.save_on_master(save_dict, os.path.join(CFG.output_dir, 'checkpoint.pth'))
        utils.save_on_master(save_dict, os.path.join(CFG.output_dir, f'checkpoint{epoch:04}.pth'))
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch}
        if utils.is_main_process():
            with (Path(CFG.output_dir) / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")
  total_time = time.time() - start_time
  total_time_str = str(datetime.timedelta(seconds=int(total_time)))
  print('Training time {}'.format(total_time_str))

In [ ]:
training_model()

## Evaluate results
Finally lets look at some random images with its most closest images according to the model. Note that at this point, we throw away the student and simply take the teacher, even though it is only the student that used gradient information directly.

In [ ]:
# hide
image_orig_data = ImageOriginalData(files)
image_orig_dl = DataLoader(
    image_orig_data,
    BATCH_SIZE*2,
    shuffle=False,
    drop_last=False,
    num_workers=4,
    pin_memory=True,
    collate_fn=CollateSingleImage(),
)

In [ ]:
teacher = teacher.eval().to(device)
embedding = []
with torch.no_grad():
    for x in tqdm(image_orig_dl):
        out = teacher(x.to(device))
        embedding.append(out.cpu())

    embedding = torch.cat(embedding, dim=0)

In [ ]:
def plot_closest_pairs(embedding, i, files):
    img = to_pil_image(resize(io.read_image(files[i])))
    plt.imshow(img)
    scores, idx = get_closest(embedding, i)

    fig, axs = plt.subplots(1, len(idx), figsize=(12, 5))
    for i, score, ax in zip(idx, scores, axs):
        img = to_pil_image(resize(io.read_image(files[i])))
        ax.imshow(img)
        ax.set_title(f"{score:.4f}")

    plt.show()

In [ ]:
i = 64
plot_closest_pairs(embedding, i, files)

In [ ]:
i = 42
plot_closest_pairs(embedding, i, files)

In [ ]:
i = 21
plot_closest_pairs(embedding, i, files)